# Concat 

In [28]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(rc={'figure.figsize':(12,6)})
plt.style.use(['seaborn-whitegrid'])

import warnings
warnings.simplefilter('ignore')

import os
import utils

Our `read_csv` function.

```python
    import pandas as pd

    def read_csv(filename):
        df = pd.read_csv(filename, skiprows=8)
        df.rename(columns={'# timestamp': 'timestamp'}, inplace=True)
        df['timestamp'] = pd.to_datetime(df['timestamp'])  #might not want this here
        df.set_index('timestamp', inplace=True)
        return df
```

In [29]:
filenames = ['data/measured_real_power.csv',
             'data/measured_reactive_power.csv']

Concat

In [30]:
df1 = utils.read_csv(filenames[0])
df2 = utils.read_csv(filenames[1])

df = pd.concat([df1, df2], axis=1) # stack side by side

In [31]:
df1.shape, df2.shape, df.shape

((12961, 20), (12961, 20), (12961, 40))

In [32]:
len(set(df.columns))

20

In [33]:
all(df1.columns == df2.columns)

True

So now we need to capture some additional information, the **prefix**

In [34]:
def read_csv(filename):
    df = pd.read_csv(filename, skiprows=8)
    df.rename(columns={'# timestamp': 'timestamp'}, inplace=True)
    df.timestamp = pd.to_datetime(df.timestamp)
    df.set_index('timestamp', inplace=True)
    prefix = os.path.basename(filename).replace('.csv','')
    df.columns = [ "{}:{}".format(prefix, x) for x in df.columns ]
    return df

df = pd.concat([ read_csv(x) for x in filenames ], axis=1)
len(set(df.columns))

40

In [35]:
df.columns

Index(['measured_real_power:triplex_meter_0',
       'measured_real_power:triplex_meter_1',
       'measured_real_power:triplex_meter_2',
       'measured_real_power:triplex_meter_3',
       'measured_real_power:triplex_meter_4',
       'measured_real_power:triplex_meter_5',
       'measured_real_power:triplex_meter_6',
       'measured_real_power:triplex_meter_7',
       'measured_real_power:triplex_meter_8',
       'measured_real_power:triplex_meter_9',
       'measured_real_power:triplex_meter_10',
       'measured_real_power:triplex_meter_11',
       'measured_real_power:triplex_meter_12',
       'measured_real_power:triplex_meter_13',
       'measured_real_power:triplex_meter_14',
       'measured_real_power:triplex_meter_15',
       'measured_real_power:triplex_meter_16',
       'measured_real_power:triplex_meter_17',
       'measured_real_power:triplex_meter_18',
       'measured_real_power:triplex_meter_19',
       'measured_reactive_power:triplex_meter_0',
       'measured_rea

## Glob 

In [36]:
import glob

Create some additional files for scaling demonstration

In [37]:
import os
import shutil

if os.path.exists("data/outputFiles"):
    shutil.rmtree("data/outputFiles")
os.mkdir("data/outputFiles")

for i in range(10):
    shutil.copyfile("data/measured_reactive_power.csv", 
                    "data/outputFiles/measured_reactive_power_{}.csv".format(i))
    shutil.copyfile("data/measured_real_power.csv", 
                    "data/outputFiles/measured_real_power_{}.csv".format(i))

In [38]:
filenames = glob.glob('data/outputFiles/*.csv')
filenames

['data/outputFiles/measured_real_power_1.csv',
 'data/outputFiles/measured_reactive_power_3.csv',
 'data/outputFiles/measured_real_power_0.csv',
 'data/outputFiles/measured_reactive_power_2.csv',
 'data/outputFiles/measured_real_power_5.csv',
 'data/outputFiles/measured_reactive_power_8.csv',
 'data/outputFiles/measured_reactive_power_4.csv',
 'data/outputFiles/measured_real_power_3.csv',
 'data/outputFiles/measured_real_power_9.csv',
 'data/outputFiles/measured_reactive_power_6.csv',
 'data/outputFiles/measured_real_power_2.csv',
 'data/outputFiles/measured_reactive_power_1.csv',
 'data/outputFiles/measured_real_power_8.csv',
 'data/outputFiles/measured_real_power_7.csv',
 'data/outputFiles/measured_reactive_power_0.csv',
 'data/outputFiles/measured_reactive_power_9.csv',
 'data/outputFiles/measured_reactive_power_7.csv',
 'data/outputFiles/measured_real_power_6.csv',
 'data/outputFiles/measured_reactive_power_5.csv',
 'data/outputFiles/measured_real_power_4.csv']

In [39]:
def read_dir(filenames):
    return pd.concat([ read_csv(x) for x in filenames ], axis=1)

read_dir(filenames).shape

(12961, 400)

Takes a while... Can we do better?

In [40]:
import cProfile

cProfile.run('read_dir(filenames)')

         98634130 function calls (98632544 primitive calls) in 68.110 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1147    0.001    0.000    0.002    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
      320    0.001    0.000    0.001    0.000 <frozen importlib._bootstrap>:103(release)
      320    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:143(__init__)
      320    0.000    0.000    0.003    0.000 <frozen importlib._bootstrap>:147(__enter__)
      320    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap>:151(__exit__)
      320    0.001    0.000    0.002    0.000 <frozen importlib._bootstrap>:157(_get_module_lock)
      320    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:176(cb)
  220/100    0.000    0.000    0.021    0.000 <frozen importlib._bootstrap>:211(_call_with_frames_removed)
     2100    0.001    0.000    0.001    0.000 <frozen importlib._bootst

       40    0.000    0.000    0.002    0.000 internals.py:4108(get)
       40    0.000    0.000    0.001    0.000 internals.py:4137(iget)
       20    0.001    0.000    0.006    0.000 internals.py:4170(delete)
       40    0.000    0.000    0.000    0.000 internals.py:4203(<genexpr>)
       20    0.002    0.000    0.006    0.000 internals.py:4208(set)
       20    0.000    0.000    0.000    0.000 internals.py:4235(value_getitem)
       40    0.000    0.000    0.000    0.000 internals.py:4287(<genexpr>)
      120    0.000    0.000    0.003    0.000 internals.py:4639(__init__)
      200    0.000    0.000    0.000    0.000 internals.py:4684(_block)
       20    0.000    0.000    0.000    0.000 internals.py:4742(external_values)
      180    0.000    0.000    0.000    0.000 internals.py:4745(internal_values)
       20    0.000    0.000    0.000    0.000 internals.py:4768(is_consolidated)
       20    0.000    0.000    0.029    0.001 internals.py:4869(create_block_manager_from_arrays)
    

In [44]:
%%writefile utils.py
import pandas as pd
import os

def read_csv(filename):
    df = pd.read_csv(filename, skiprows=8)
    df.rename(columns={'# timestamp': 'timestamp'}, inplace=True)
    #df.timestamp = pd.to_datetime(df.timestamp) #hold off for now...
    df.set_index('timestamp', inplace=True)
    prefix = os.path.basename(filename).replace('.csv','')
    df.columns = [ "{}:{}".format(prefix, x) for x in df.columns ] 
    return df

def read_dir(filenames):
    df = pd.concat([ read_csv(x) for x in filenames ], axis=1, sort=False)
    df.index = pd.to_datetime(df.index)
    return df

Overwriting utils.py


In [45]:
%load_ext autoreload
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
%autoreload 2
import utils

df = utils.read_dir(filenames)
df.shape

(12961, 400)

In [47]:
df.head()

,measured_real_power_1:triplex_meter_0,measured_real_power_1:triplex_meter_1,measured_real_power_1:triplex_meter_2,measured_real_power_1:triplex_meter_3,measured_real_power_1:triplex_meter_4,measured_real_power_1:triplex_meter_5,measured_real_power_1:triplex_meter_6,measured_real_power_1:triplex_meter_7,measured_real_power_1:triplex_meter_8,measured_real_power_1:triplex_meter_9,...,measured_real_power_4:triplex_meter_10,measured_real_power_4:triplex_meter_11,measured_real_power_4:triplex_meter_12,measured_real_power_4:triplex_meter_13,measured_real_power_4:triplex_meter_14,measured_real_power_4:triplex_meter_15,measured_real_power_4:triplex_meter_16,measured_real_power_4:triplex_meter_17,measured_real_power_4:triplex_meter_18,measured_real_power_4:triplex_meter_19
timestamp,,,,,,,,,,,,,,,,,,,,,
2013-07-01 00:00:00,510.912,510.912,510.912,547.924,510.912,510.912,547.924,547.924,581.637,581.637,...,384.146,384.146,384.146,531.494,531.494,531.494,530.845,530.845,271.123,383.677
2013-07-01 00:01:00,510.912,510.912,510.912,547.924,510.912,510.912,547.924,547.924,581.637,581.637,...,384.146,384.146,384.146,531.494,531.494,531.494,530.845,530.845,271.123,383.677
2013-07-01 00:02:00,510.912,510.912,510.912,547.924,510.912,510.912,547.924,547.924,581.637,581.637,...,384.146,384.146,384.146,531.494,531.494,531.494,530.845,530.845,271.123,383.677
2013-07-01 00:03:00,510.912,510.912,510.912,547.924,510.912,510.912,547.924,547.924,581.637,581.637,...,384.146,384.146,384.146,531.494,531.494,531.494,530.845,530.845,271.123,383.677
2013-07-01 00:04:00,510.911,510.911,510.911,547.922,510.911,510.911,547.922,547.922,581.645,581.645,...,384.151,384.151,384.151,384.151,531.501,531.501,530.849,530.849,271.125,383.680


## The stacked method 

In [48]:
filename = 'data/measured_real_power.csv'

df = pd.read_csv(filename, skiprows=8)
df.rename(columns={'# timestamp': 'timestamp'}, inplace=True)
df = df.melt(id_vars='timestamp')
df.head()

,timestamp,variable,value
0,2013-07-01 00:00:00 UTC,triplex_meter_0,510.912
1,2013-07-01 00:01:00 UTC,triplex_meter_0,510.912
2,2013-07-01 00:02:00 UTC,triplex_meter_0,510.912
3,2013-07-01 00:03:00 UTC,triplex_meter_0,510.912
4,2013-07-01 00:04:00 UTC,triplex_meter_0,510.911


In [49]:
prefix = os.path.basename(filename).replace('.csv','')
df['attribute'] = prefix
df.head()

,timestamp,variable,value,attribute
0,2013-07-01 00:00:00 UTC,triplex_meter_0,510.912,measured_real_power
1,2013-07-01 00:01:00 UTC,triplex_meter_0,510.912,measured_real_power
2,2013-07-01 00:02:00 UTC,triplex_meter_0,510.912,measured_real_power
3,2013-07-01 00:03:00 UTC,triplex_meter_0,510.912,measured_real_power
4,2013-07-01 00:04:00 UTC,triplex_meter_0,510.911,measured_real_power


We can stack this with concat

In [50]:
%%writefile utils.py
import pandas as pd
import os

def read_csv(filename):
    df = pd.read_csv(filename, skiprows=8)
    df.rename(columns={'# timestamp': 'timestamp'}, inplace=True)
    df = df.melt(id_vars='timestamp')
    prefix = os.path.basename(filename).replace('.csv','')
    df['attribute'] = prefix
    return df

def read_dir(filenames):
    df = pd.concat([ read_csv(x) for x in filenames ], axis=0, sort=False)  # axis = 0 for stacking
    #df.index = pd.to_datetime(df.index)
    return df

Overwriting utils.py


In [51]:
%autoreload 2
import utils

df = utils.read_dir(filenames)
df.shape

(5184400, 4)

In [52]:
# takes a really long time....
#df.timestamp = pd.to_datetime(df.timestamp)

Reshaping can help...

In [53]:
tmp = df.pivot_table(index='timestamp', columns=['variable', 'attribute'], values='value')
print(tmp.shape)
df.index = pd.to_datetime(df.index)
final = tmp.reset_index().melt(id_vars='timestamp')
final.shape

(12961, 400)


(5184400, 4)

In [55]:
%%writefile utils.py
import pandas as pd
import os

def read_csv(filename):
    df = pd.read_csv(filename, skiprows=8)
    df.rename(columns={'# timestamp': 'timestamp'}, inplace=True)
    df = df.melt(id_vars='timestamp')
    prefix = os.path.basename(filename).replace('.csv','')
    df['attribute'] = prefix
    return df

def read_dir(filenames):
    df = pd.concat([ read_csv(x) for x in filenames ], axis=0, sort=False)  # axis = 0 for stacking
    #df.index = pd.to_datetime(df.index)
    tmp = df.pivot_table(index='timestamp', columns=['variable', 'attribute'], values='value')
    tmp.index = pd.to_datetime(tmp.index)
    final = tmp.reset_index().melt(id_vars='timestamp')
    return final

Overwriting utils.py
